# Tratamiento de Variables Categóricas en Python

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('../datasets/ecom-expense/Ecom Expense.csv')

### Necesitamos crear 2 variables dummy, una para el sexo, y otra para el tier de la ciudad

In [3]:
dummy_gender = pd.get_dummies(data['Gender'], prefix='Gender')
dummy_cityTier = pd.get_dummies(data['City Tier'], prefix='City')

In [4]:
new_data = data.join([dummy_gender, dummy_cityTier])

### Vamos a crear el set de variables predictoras X

In [5]:
feature_cols = ['Monthly Income', 'Transaction Time', 
                'Gender_Male', 'Gender_Female',
                'City_Tier 1', 'City_Tier 2', 'City_Tier 3',
                'Record']
X = pd.DataFrame(new_data[feature_cols])
X.head()

,Monthly Income,Transaction Time,Gender_Male,Gender_Female,City_Tier 1,City_Tier 2,City_Tier 3,Record
0,7313,627.668127,0,1,1,0,0,5
1,17747,126.904567,0,1,0,1,0,3
2,22845,873.469701,1,0,0,1,0,2
3,18552,380.219428,0,1,1,0,0,7
4,14439,403.374223,0,1,0,1,0,2


### Creamos ahora el set de la variable a predecir Y

In [6]:
Y = new_data['Total Spend']

In [7]:
lm = LinearRegression()
lm.fit(X, Y)
print(f'Bias = {lm.intercept_}')
print(f'Coefficients = {lm.coef_}')
print('---------------------------------------')
print(f'R2 = {lm.score(X, Y)}')

Bias = -79.41713030137271
Coefficients = [ 1.47538980e-01  1.54946125e-01  1.31025013e+02 -1.31025013e+02
  7.67643260e+01  5.51389743e+01 -1.31903300e+02  7.72233446e+02]
---------------------------------------
R2 = 0.9179923586131016


**Hay una forma de visualizar mejor los coeficientes, que es con la función zip**

In [8]:
list(zip(feature_cols, lm.coef_))

[('Monthly Income', 0.1475389804920574),
 ('Transaction Time', 0.15494612549589615),
 ('Gender_Male', 131.02501325554562),
 ('Gender_Female', -131.02501325554562),
 ('City_Tier 1', 76.76432601049517),
 ('City_Tier 2', 55.13897430923272),
 ('City_Tier 3', -131.90330031972783),
 ('Record', 772.2334457445645)]

### Podemos comprobar que la suma de los coeficientes de las variables dummy da cero tanto para el sexo como para el city tier

### Las variables dummy pueden simplificarse, esto se hace eliminando una de ellas.
### Esto puede hacerse debido a que la eliminada puede inferirse de las otras, esto se vé muy claro en variables categóricas binarias como las de Sexo

In [9]:
gender_dummies = pd.get_dummies(data['Gender'], prefix='Gender').iloc[:, :-1]
gender_dummies.head()

,Gender_Female
0,1
1,1
2,0
3,1
4,1


### Si el Gender_Female es 1, es fácil inferir que el Gender_Male será 0, ya que no puede tener los 2 sexos a la vez.
### Por ello lo que hacemos directamente es eliminar una de estas columnas ya que no está aportando información nueva
### Lo mismo para el City Tier

In [10]:
cityTier_dummies = pd.get_dummies(data['City Tier'], prefix='City').iloc[:, :-1]
cityTier_dummies.head()

,City_Tier 1,City_Tier 2
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1


### Vamos a volver a crear el modelo pero con las variables dummies simplificadas

In [11]:
new_data_2 = data.join([gender_dummies, cityTier_dummies])
new_data_2.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,City_Tier 1,City_Tier 2
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,1
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,1


In [12]:
feature_cols = ['Monthly Income', 'Transaction Time', 
                'Gender_Female',
                'City_Tier 1', 'City_Tier 2',
                'Record']
X = new_data_2[feature_cols]
X.head()

,Monthly Income,Transaction Time,Gender_Female,City_Tier 1,City_Tier 2,Record
0,7313,627.668127,1,1,0,5
1,17747,126.904567,1,0,1,3
2,22845,873.469701,0,0,1,2
3,18552,380.219428,1,1,0,7
4,14439,403.374223,1,0,1,2


In [13]:
lm2 = LinearRegression()
lm2.fit(X, Y)
print(f'Bias = {lm2.intercept_}')
print(f'Coefficients = {lm2.coef_}')
print('---------------------------------------')
print(f'R2 = {lm2.score(X, Y)}')

Bias = -80.29541736554984
Coefficients = [ 1.47538980e-01  1.54946125e-01 -2.62050027e+02  2.08667626e+02
  1.87042275e+02  7.72233446e+02]
---------------------------------------
R2 = 0.9179923586131016


### El modelo rinde igual de bien que antes de simplificar, pero ahora, ahorramos en recursos al entrenarlo y testearlo con un dataset reducido